https://chatgpt.com/share/687d132f-70f0-8003-8800-16c45376f179

i extract subtitle from youtube video
the video is 2hours long
i want summarize with Key points of the video using llm model
so i chunk subtitle every 10 minutes and input each chunk to model to summarize
eg. 1 time summarize for 1 chunk
and then I combine all the sumarize together
I use this method because llm model seem to cunfuse when I input the large amount of data into it
so I chunk it first and do all above
is it a good method or any method suggest that better than this? 

to do
- try chunking in token 
    - 1,000–2,000 tokens per chunk is the safe sweet spot
    - Add 10–20% overlap to catch mid-topic cuts

In [3]:
import requests
from bs4 import BeautifulSoup

url = "https://www.youtube.com/watch?v=H8S9xg8iYuc&t=11s"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
title = soup.title.string.replace(" - YouTube", "").strip()

In [4]:
import re

# get video title name
def clean_title(text):
    return re.sub(r'[^0-9a-zA-Z\u0E00-\u0E7F\.]', '', text)
title = clean_title(text=title)

# get video id
match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
if match:
    video_id = match.group(1)

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi

# Fetch transcript (auto-captions or uploaded)
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['th', 'en'])

# Optionally, save to file
file_name = f'{title}_{video_id}'
with open(f"{file_name}_subtitle.txt", "w", encoding="utf-8") as f:
    for entry in transcript:
        f.write(f"{entry['start']:.2f}s: {entry['text']}\n")
        # f.write(f"{entry['text']}\n")

In [14]:
import re

def load_subtitles(file_path):
    subtitles = []
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            match = re.match(r'([0-9.]+)s:\s(.+)', line.strip())
            if match:
                start_time = float(match.group(1))
                text = match.group(2)
                subtitles.append({'start': start_time, 'text': text})
    return subtitles

def chunk_subtitles(subtitles, chunk_size=60, overlap=20):
    """
    Chunk subtitles into segments of `chunk_size` seconds with `overlap` seconds.
    """
    chunks = []
    max_time = subtitles[-1]['start']
    start_time = 0

    while start_time <= max_time:
        end_time = start_time + chunk_size
        chunk_text = []
        for entry in subtitles:
            if start_time <= entry['start'] < end_time:
                chunk_text.append(entry['text'])
        if chunk_text:
            chunks.append({
                'start': start_time,
                'end': end_time,
                'text': ' '.join(chunk_text)
            })
        start_time += chunk_size - overlap
    return chunks

subtitles = load_subtitles(f"{file_name}_subtitle.txt")
# chunk_dict = chunk_subtitles(subtitles, chunk_size=600, overlap=60)
chunk_dict = chunk_subtitles(subtitles, chunk_size=300, overlap=30)

In [15]:
chunk_dict[0]

{'start': 0,
 'end': 300,
 'text': '[เพลง] ค่ะสวัสดีนักลงทุนทุกท่านนะคะก็สำหรับผล ประกอบ การของ Opportunity  Day ของบริษัทปลุก ผักพรรักแม่ในวันนี้นะคะเราได้รับเกียรติ จากผู้บริหารทั้ง 3 ท่านค่ะท่านแรกก็จะ เป็นคุณชรานะคะ CFO  CEO ค่ะสวัสดีครับ ค่ะท่านที่ 2 คุณวิทเพ็ญ CFO ค่ะสดีค่ะ ท่านที่ 3 คุณจิรายุทธ CAO ค่ะสวัสดีครับ ค่ะก็วันนี้ทั้ง 3 ท่านจะให้เกียรติเราใน การพีเซนในวันนี้นะคะสำหรับ Agenda เนี่ย เรื่องแรกก็จะเป็น Business อัปเดตนะคะ เดี๋ยวจะอัปเดตโดยคุณชลากรนะคะแล้วก็  Business  Outlook ก็เป็นคุณชลากรนะคะ ค่ะก็เดี๋ยวจะขอเริ่มที่ไซส์แรกเลยแล้ว กันค่ะ ครับผมก็ในส่วนของตัวอ่าอัปเดตนะครับ ไตรมาส 1 ของปี 68 เนี่ยนะครับเราก็ทำได้ ดีขึ้นนะครับทั้งในส่วนของตัวยอดขายกำไร สุทธิแล้วก็ตัวมาจinนะครับยอดขายเนี่ย เติบโต 33% นะครับจากปีก่อนจากการเติบโต จากทั้งของทุกแบรนด์นะครับในส่วนของตัว โอ้เองที่มีการขยายสาขาเพิ่มขึ้น 7 สาขาในปีที่แล้วแล้วก็แบรนด์ใหม่ที่เปิด ตัวในปีที่แล้วทั้งโจแล้วก็ Rap  and  R นะครับแล้วก็ตัว SSG เนี่ยอยู่ที่เติบโต อยู่ที่ 0.1%  1% ครับผมในส่วนที่ 2 เนี่ยก็คือตัวเปอร์เซ็นต์ GP

In [8]:
# # subtitle = [subtitle['text'] for subtitle in subtitles]
# with open(f"{file_name}_subtitle.txt", 'r') as f:
#     contents = f.read()

In [9]:
title = file_name.split('_')[0]

## Use “Key points extraction” instead of general summary
## Use hierarchical summarization

In [11]:
# === PROMPTS ===
KEY_POINTS_PROMPT = """
You are a helpful assistant.
Below is a section of a video transcript.
Extract the key points using clear bullet points.
Label each point with one of these: [Fact], [Idea], [Quote], [Question], [Action].
Be precise. Keep points short and factual.
Do not add commentary or extra text.

Transcript:
\"\"\"
{chunk_text}
\"\"\"
"""

SECTION_MERGE_PROMPT = """
Below are key points extracted from multiple chunks of a section.
Merge them into a single list:
- Remove duplicates or near-duplicates.
- Group similar ideas if needed.
- Keep clear bullet points.
- Preserve labels like [Fact], [Idea], etc.

Key Points:
\"\"\"
{key_points_text}
\"\"\"
"""

FINAL_SUMMARY_PROMPT = """
Below are the section-level key points for the entire video.
Write a single, well-organized summary:
- Merge and deduplicate.
- Organize by themes.
- Keep [Fact], [Idea], [Quote], [Action], etc.
- Present clearly for a slide deck or notes.
- Do not add fluff.

All Section Points:
\"\"\"
{all_sections_text}
\"\"\"
"""

# === HELPERS ===

import ollama
def call_llm(prompt: str):
    response = ollama.chat(
        model="llama3",
        messages=[
            {"role": "system", "content": "You are a precise summarization assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["message"]["content"]

In [16]:
# === PIPELINE ===

chunk_key_points = []
print("Extracting key points for each chunk...")
for i, chunk in enumerate(chunk_dict):
    prompt = KEY_POINTS_PROMPT.format(chunk_text=chunk['text'])
    result = call_llm(prompt)
    print(f"Chunk {i+1} done.")
    chunk_key_points.append(result)

# Group into sections, e.g., 3 chunks per section
SECTION_SIZE = 3
section_key_points = []

print("\nMerging chunks into sections...")
for i in range(0, len(chunk_key_points), SECTION_SIZE):
    group = chunk_key_points[i:i+SECTION_SIZE]
    group_text = "\n\n".join(group)
    prompt = SECTION_MERGE_PROMPT.format(key_points_text=group_text)
    result = call_llm(prompt)
    print(f"Section {i//SECTION_SIZE + 1} done.")
    section_key_points.append(result)

# Final merge
print("\nMerging all sections into final summary...")
all_sections_text = "\n\n".join(section_key_points)
final_prompt = FINAL_SUMMARY_PROMPT.format(all_sections_text=all_sections_text)
final_summary = call_llm(final_prompt)

print("\n✅ Final Summary:\n")
print(final_summary)

Extracting key points for each chunk...
Chunk 1 done.
Chunk 2 done.
Chunk 3 done.
Chunk 4 done.
Chunk 5 done.
Chunk 6 done.
Chunk 7 done.
Chunk 8 done.

Merging chunks into sections...
Section 1 done.
Section 2 done.
Section 3 done.

Merging all sections into final summary...

✅ Final Summary:

Here is a well-organized summary of the section-level key points:

**Company Performance**

* Fact: Opportunity Day was held by บริษัทปลุก ผักพรรักแม่, with revenue growing 33% YoY in Q1 68.
* Fact: Same Store Sales Growth (SSSG) was 0.1-1%, and same store sales declined by 2 digits in Q2 68 due to external factors.

**Growth Strategy**

* Idea: The company's growth was driven by new store openings and the launch of new brands, with plans to open more stores in the future.
* Action: OKJ aims to become a leading player in the Thai food delivery market by offering unique and high-quality products.
* Fact: The company has received positive feedback from customers and is working to improve its servi

In [17]:
with open(f"{file_name}_summary_v2.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)

## Try Map-Reduce with embeddings (advanced)